## Initialize the ckan environment and requests session

In [ ]:
%%bash
python3 -m pip -q install -U \
    https://github.com/OriHoch/datapackage-pipelines-ckan/archive/support-multiple-datasets-headers-translated-error-message.zip \
    https://github.com/OriHoch/ckanext-datapackage_pipelines/archive/master.zip#subdirectory=datapackage_pipelines_ckanext \
    'dataflows'

## Fetch all the ckan users and groups

In [ ]:
from dataflows import Flow, printer, checkpoint
from importlib import reload
from ckan import ckan

!{'rm -rf .checkpoints/all_groups_orgazniations'}

Flow(
    ckan('get', 'organizations'),
    ckan('get', 'groups'),
    printer(tablefmt='html', num_rows=1),
    checkpoint('all_groups_orgazniations')
).process()[1]

## Get parameters for create member

In [ ]:
from dataflows import Flow, checkpoint, printer

def get_group_users_to_create(entities, all_organization_user_ids):
    for entity in entities:
        entity_user_ids = [entity_user['id'] for entity_user in entity['users']]
        for organiztion_user_id in all_organization_user_ids:
            if organiztion_user_id not in entity_user_ids:
                yield {'object': organiztion_user_id,
                       'object_type': 'user',
                       'id': entity['id'],
                       'capacity': 'member'}
                

def get_create_member_params(package):
    package.pkg.remove_resource('ckan_organizations')
    package.pkg.remove_resource('ckan_groups')
    package.pkg.add_resource({"name": "group_users_to_create", "path": "group_users_to_create.csv",
                              "schema": {"fields": [{"name": "id", "type": "string"},
                                                    {"name": "object", "type": "string"},
                                                    {"name": "object_type", "type": "string"},
                                                    {"name": "capacity", "type": "string"}]}})
    yield package.pkg
    all_organization_user_ids = set()
    all_group_entities_user_ids = {}
    for rows in package:
        if rows.res.name == 'ckan_organizations':
            for org in rows:
                for org_user in org['users']:
                    all_organization_user_ids.add(org_user['id'])
        elif rows.res.name == 'ckan_groups':
            yield get_group_users_to_create(rows, all_organization_user_ids)
        else:
            yield rows

!{'rm -rf .checkpoints/group_users_to_create'}
    
Flow(
    checkpoint('all_groups_orgazniations'),
    get_create_member_params,
    printer(tablefmt='html', num_rows=1),
    checkpoint('group_users_to_create')
).process()[1]

## Create the group users memberships

In [ ]:
from dataflows import Flow, checkpoint, printer
from ckan import ckan

Flow(
    checkpoint('group_users_to_create'),
    ckan('create', 'member'),
    printer(tablefmt='html', num_rows=1)
).process()[1]
